# ------------------------------ Bulletin de Paie Format1 ------------------------------ # 

## 1- Extraction texte d'un PDF

### 1e méthode - PDFMiner

In [ ]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO

def get_pdf_file_content(path_to_pdf):
    
    resource_manager = PDFResourceManager(caching=True)
    out_text = StringIO()

    codec = 'utf-8'
    laParams = LAParams()
    
    text_converter = TextConverter(resource_manager, out_text, laparams=laParams)
    fp = open(path_to_pdf, 'rb')
    

    interpreter = PDFPageInterpreter(resource_manager, text_converter)
    for page in PDFPage.get_pages(fp, pagenos=set(), maxpages=0, password="", caching=True, check_extractable=True):
        interpreter.process_page(page)
    text = out_text.getvalue()

    
    fp.close()
    text_converter.close()
    out_text.close()

    return text

path_to_pdf = "EXMPLE-FP-WORD_EX4[2140].pdf"
input_text = (get_pdf_file_content(path_to_pdf))
print(input_text)


## 2 - sauvegarde en fichier texte 

In [ ]:
from io import StringIO

f = StringIO(input_text)
print(f.getvalue())

## 3 - Extraction spécifiques données

### 1e méthode : liste et index

In [ ]:
from io import StringIO
from pdfminer.high_level import extract_text


input_text= extract_text('EXMPLE-FP-WORD_EX4[2140].pdf')
f = StringIO(input_text)

mylist = [line.rstrip('\n') for line in f]
while '' in mylist:      
    mylist.remove('')
#print(mylist)

      
date_index = mylist.index("Paye du : ")
a= (mylist[date_index+1])
b=  str(a).strip('[]') #list to str pour prendre substring
mois=(b[3:5])
print(mois) 

i = mylist.index("FONTENAY-SOUS-BOIS ")
nom = (mylist[i+3])
print(nom)
index_nom = mylist.index(nom) #chercher l'index du nom 

adresse_index = mylist.index('Total brut   ')
adresse = (mylist[index_nom+1:adresse_index]) #va donner plusieurs items d'une liste
adresse = " ".join(adresse) #pour affichage meilleur 
print(adresse)

net_index = mylist.index('NET PAYE EN EUROS ')
net= mylist[net_index+2]
print(net)

total_index= mylist.index('CONGES ')
total = (mylist[total_index-1])
print(total)

f.close()


## 4 - Mettre résultat sous CSV & JSON

### csv


In [ ]:
import pandas as pd
from csv import DictWriter 
from io import StringIO
from pdfminer.high_level import extract_text


input_text= extract_text('EXMPLE-FP-WORD_EX4[2140].pdf')
f = StringIO(input_text)

mylist = [line.rstrip('\n') for line in f]
while '' in mylist:      
    mylist.remove('')
#print(mylist)

        
i = mylist.index("FONTENAY-SOUS-BOIS ")
nom = (mylist[i+3])
index_nom = mylist.index(nom)

date_index = mylist.index("Paye du : ")
a= (mylist[date_index+1])
b=  str(a).strip('[]') #list to str pour prendre substring
mois=(b[3:5]) 


adresse_index = mylist.index('Total brut   ')
adresse = (mylist[index_nom+1:adresse_index]) #va donner plusieurs items d'une liste
adresse = " ".join(adresse) #pour affichage meilleur, list to str 

net_index = mylist.index('NET PAYE EN EUROS ')
net= mylist[net_index+2]  

total_index= mylist.index('CONGES ')
total = (mylist[total_index-1])

#df = pd.DataFrame(columns=['Mois','Nom','Adresse','Net_payé','Total_versé'])
new = {'Mois': mois, 'Nom': nom, 'Adresse': adresse,'Net_payé':net, 'Total_versé':total}
#df =df.append(new, ignore_index=True)

field_names = ['Mois','Nom','Adresse','Net_payé','Total_versé']

with open('paie.csv', 'a+', newline ='') as f_object:
    csv_writer = DictWriter(f_object,fieldnames=field_names )
    if f_object.tell() == 0:
            csv_writer.writeheader()
    csv_writer.writerow(new) 
    f_object.close()

df= pd.read_csv('paie.csv', encoding='latin-1')
#df.to_json('azra.json',orient='records')
df.head()

### JSON

In [ ]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO
import pandas as pd 
import csv
from csv import DictWriter
import json 


##PRENDRE CONTENU DU PDF
def get_pdf_file_content(path):
    
    resource_manager = PDFResourceManager(caching=True)
    out_text = StringIO()
    codec = 'utf-8'
    laParams = LAParams()
    text_converter = TextConverter(resource_manager, out_text, laparams=laParams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(resource_manager, text_converter)
    for page in PDFPage.get_pages(fp, pagenos=set(), maxpages=0, password="", caching=True, check_extractable=True):
        interpreter.process_page(page)
    text = out_text.getvalue()
    
    fp.close()
    text_converter.close()
    out_text.close()

    return text

##EXTRACTION 
def create_extract_file(path): #créer un fichier file et extrait les données
    
    input_text = get_pdf_file_content(path)
    with open("fichier.txt",'w',encoding='utf-8') as f:
        f.write(input_text)
    with open("fichier.txt",'r',encoding='utf-8') as f:
        mylist = [line.rstrip('\n') for line in f]
        while '' in mylist:      
            mylist.remove('')
    
        
    date_index = mylist.index("Paye du : ")
    a= (mylist[date_index+1])
    b=  str(a).strip('[]') #list to str pour prendre substring
    mois=(b[3:5])
    print('mois: ',mois) 
 
    i = mylist.index("FONTENAY-SOUS-BOIS ")
    nom = (mylist[i+3])
    print('nom: ',nom)
    index_nom = mylist.index(nom) #chercher l'index du nom 
    
    adresse_index = mylist.index('Total brut   ')
    adresse = (mylist[index_nom+1:adresse_index]) #va donner plusieurs items d'une liste
    adresse = " ".join(adresse) #pour affichage meilleur 
    print('adresse: ',adresse)
    
    net_index = mylist.index('NET PAYE EN EUROS ')
    net= mylist[net_index+2]
    print('net: ',net)

    total_index= mylist.index('CONGES ')
    total = (mylist[total_index-1])
    print('total: ',total)
    
   
    f.close()



    jsonArray = []
      
    #read csv file
    with open('paie.csv', encoding='latin-1') as csvf: 
        #load csv file data using csv library's dictionary reader
        csvReader = csv.DictReader(csvf) 

        #convert each csv row into python dict
        for row in csvReader: 
            #add this python dict to json array
            jsonArray.append(row)
  
    #convert python jsonArray to JSON String and write to file
    with open('test.json', 'w', encoding='latin-1') as jsonf: 
        jsonString = json.dumps(jsonArray, indent=4)
        jsonf.write(jsonString)

path = "EXMPLE-FP-WORD_EX4[2140].pdf"
input_text = get_pdf_file_content(path)
create_extract_file(path)


# --------------- Fonction entière finale ---------------

In [ ]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO
import pandas as pd 
import csv
from csv import DictWriter
import json 


#------------ PRENDRE CONTENU PDF ------------#
def get_pdf_file_content(path):
    
    resource_manager = PDFResourceManager(caching=True)
    out_text = StringIO()
    codec = 'utf-8'
    laParams = LAParams()
    text_converter = TextConverter(resource_manager, out_text, laparams=laParams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(resource_manager, text_converter)
    for page in PDFPage.get_pages(fp, pagenos=set(), maxpages=0, password="", caching=True, check_extractable=True):
        interpreter.process_page(page)
    text = out_text.getvalue()
    
    fp.close()
    text_converter.close()
    out_text.close()

    return text

#------------ EXTRACTION ------------#
def create_extract_file(input_text): #créer un fichier file et extrait les données
    
    #Ouverture fichier et mise en forme liste 
    f = StringIO(input_text)

    mylist = [line.rstrip('\n') for line in f]
    while '' in mylist:      
        mylist.remove('')
    #print(mylist)

    
     
    #Extraction mots 
    date_index = mylist.index("Paye du : ")
    a= (mylist[date_index+1])
    b=  str(a).strip('[]') #list to str pour prendre substring
    mois=(b[3:5])
 
    i = mylist.index("FONTENAY-SOUS-BOIS ")
    nom = (mylist[i+3])
    index_nom = mylist.index(nom) #chercher l'index du nom 
    
    adresse_index = mylist.index('Total brut   ')
    adresse = (mylist[index_nom+1:adresse_index]) #va donner plusieurs items d'une liste
    adresse = " ".join(adresse) #pour affichage meilleur 
    
    net_index = mylist.index('NET PAYE EN EUROS ')
    net= mylist[net_index+2]

    total_index= mylist.index('CONGES ')
    total = (mylist[total_index-1])

#------------ CREATION CSV ------------#

    new = {'Mois': mois, 'Nom': nom, 'Adresse': adresse,'Net_paye':net, 'Total_verse':total}
    field_names = ['Mois','Nom','Adresse','Net_paye','Total_verse']

    with open('paie.csv', 'a+', newline ='') as f_object:
        csv_writer = DictWriter(f_object,fieldnames=field_names )
        if f_object.tell() == 0:
                csv_writer.writeheader()
        csv_writer.writerow(new) 
        f_object.close()
    
    f.close()
   
    print('nom: ', nom)
    print('mois: ',mois)
    print('adresse: ',adresse)
    print('net: ', net)
    print('total: ',total) 
    
        
#------------ CREATION JSON ------------#

def csv_to_json(jsonFilePath):
    jsonArray = []
      
    #read csv file
    with open('paie.csv', encoding='latin-1') as csvf: 
        #load csv file data using csv library's dictionary reader
        csvReader = csv.DictReader(csvf) 

        #convert each csv row into python dict
        for row in csvReader: 
            #add this python dict to json array
            jsonArray.append(row)
  
    #convert python jsonArray to JSON String and write to file
    with open(jsonFilePath, 'w', encoding='latin-1') as jsonf: 
        jsonString = json.dumps(jsonArray, indent=4)
        jsonf.write(jsonString)

    
    
def main():
    
    path = input("entrer le fichier pdf à extraire sous format file.pdf : ")
    input_text = get_pdf_file_content(path)
    print('\n --- conversion texte des données ---')
    print('--- extraction des données --- \n')
    create_extract_file(input_text)
    print('\n --- création "paie.csv", voir répertoire où ce fichier se trouve--- \n')
    json= input('entrer le nom du json file à créer sous format file.json :  ')
    csv_to_json(json)
    print('--- création fichier json',json,'fait --- \n')
    



    
    


if __name__ == "__main__":
    main()

# ------------------------------ Bulletin de Paie Format2  ------------------------------ # 

In [ ]:
from io import StringIO
from pdfminer.high_level import extract_text


input_text= extract_text('fiche_individuelle_A1221.pdf')
with open("format2.txt", "w", encoding="utf-8") as datafile:
    print(input_text, file=datafile) # this goes to the opened file.
    
f = open("format2.txt", encoding = 'utf-8').read()
mylist=f.split('\n')
print(mylist)




### URSAFF

In [ ]:
#URSAFF
urssaf01i = mylist.index('Base Contribution solidarité autonomie')
urssaf01_1= (mylist[urssaf01i+19:urssaf01i+42])

urssaf01_2i = mylist.index('CSG déductible (TOT)')
urssaf01_2 = (mylist[urssaf01_2i+4:urssaf01_2i+19])

urssaf01_3 = (mylist[urssaf01_2i+38:urssaf01_2i+41])

urssaf01_4 = (mylist[urssaf01_2i+48:urssaf01_2i+51])

urssaf01_5i = mylist.index('Net à payer')
urssaf01_5 = (mylist[urssaf01_5i+11:urssaf01_5i+14])



### RETRAITE

In [ ]:
#Retraite complémentaire 
retraite01_1i = mylist.index('CSG déductible (TOT)')
retraite01 = (mylist[retraite01_1i+19:retraite01_1i+23])
retraite01


### MUTUELLE ET PREVOYANCE

In [ ]:
mutuelle01_1i = mylist.index('CSG déductible (TOT)')
mutuelle01 = (mylist[mutuelle01_1i+27:mutuelle01_1i+38])
mutuelle01

### TAXE

In [ ]:
taxe01_i = mylist.index('CSG déductible (TOT)')
taxe01 = (mylist[taxe01_i+41:taxe01_i+47])
taxe01

### SUM LIST CATEGORIES

In [ ]:
#URSAFF
urssaf01i = mylist.index('Base Contribution solidarité autonomie')
urssaf01_1= (mylist[urssaf01i+19:urssaf01i+42])

urssaf01_2i = mylist.index('CSG déductible (TOT)')
urssaf01_2 = (mylist[urssaf01_2i+4:urssaf01_2i+19])

urssaf01_3 = (mylist[urssaf01_2i+38:urssaf01_2i+41])

urssaf01_4 = (mylist[urssaf01_2i+48:urssaf01_2i+51])

urssaf01_5i = mylist.index('Net à payer')
urssaf01_5 = (mylist[urssaf01_5i+11:urssaf01_5i+14])


def sub_sum(liste):
    #replace , by . to convert str to float 
    prov = []
    for i in liste: 
        if i != ' ':
            a = i.replace(",", ".")
            prov.append(a)

    #remove space, else we can't convert str to float 
    prov2 = []
    for i in prov:     
            a = i.replace(" ", "")
            prov2.append(a)

    #convert list str to float list 
    liste=([float(x) for x in prov2])
    
    #sub sum
    liste=sum(liste)
    
    return(liste)
    
def sum(liste):
    a = sub_sum(liste)
    b = sub_sum(liste)
    c = sub_sum(liste)
    d = sub_sum(liste)
    e = sub_sum(liste)
    
    return a,b,c,d,e

def main():
    
    sum(urssaf1)


    sum= a+b+c+d+e
    print(sum)

if __name__ == "__main__":
    main()

### Nom

In [ ]:
#nom
nom_i= mylist.index("FICHES  INDIVIDUELLES  DETAILLEES")
nom = (mylist[nom_i+9])
nom

# --------------- Fonction entière finale ---------------

In [4]:
from io import StringIO
from pdfminer.high_level import extract_text
import pandas as pd
import csv
from csv import DictWriter

#-----------LOAD-----------#
def load(path):
    input_text= extract_text(path)
    f = StringIO(input_text)
    mylist = (f.getvalue())
    mylist = mylist.split("\n")
    return (mylist)



#----------INDEX----------#
#ursaff divisé en 5 parties
def ursaff1(mylist):
    #URSSAFF
    urssaf01i = mylist.index('Base Contribution solidarité autonomie')
    urssaf01_1= (mylist[urssaf01i+19:urssaf01i+42])
    return urssaf01_1
    
def ursaff2(mylist):
    urssaf01_2i = mylist.index('CSG déductible (TOT)')
    urssaf01_2 = (mylist[urssaf01_2i+4:urssaf01_2i+19])
    return urssaf01_2


def ursaff3(mylist):
    urssaf01_2i = mylist.index('CSG déductible (TOT)')
    urssaf01_3 = (mylist[urssaf01_2i+38:urssaf01_2i+41])
    return urssaf01_3

def ursaff4(mylist):
    urssaf01_2i = mylist.index('CSG déductible (TOT)')
    urssaf01_4 = (mylist[urssaf01_2i+48:urssaf01_2i+51])
    return urssaf01_4
    
    
def ursaff5(mylist):
    urssaf01_5i = mylist.index('Net à payer')
    urssaf01_5 = (mylist[urssaf01_5i+11:urssaf01_5i+14])
    return urssaf01_5

def retraite1(mylist):
    #Retraite complémentaire 
    retraite01_1i = mylist.index('CSG déductible (TOT)')
    retraite01 = (mylist[retraite01_1i+19:retraite01_1i+23])
    return retraite01
    
def mutuelle1(mylist):
    #mutuelle
    mutuelle01_1i = mylist.index('CSG déductible (TOT)')
    mutuelle01 = (mylist[mutuelle01_1i+27:mutuelle01_1i+38])
    return mutuelle01
    
def taxe1(mylist): 
    #taxe 
    taxe01_i = mylist.index('CSG déductible (TOT)')
    taxe01 = (mylist[taxe01_i+41:taxe01_i+47])
    return taxe01

#nom
def nom1(mylist):
    nom_i= mylist.index("FICHES  INDIVIDUELLES  DETAILLEES")
    nom = (mylist[nom_i+9])
    nom
    return nom

#----------SOMME DANS MEME CATEGORIE--------------#
def sub_sum(liste): #additionne une même liste 
    #replace , by . to convert str to float 
    prov = []
    for i in liste: 
        if i != ' ':
            a = i.replace(",", ".")
            prov.append(a)

    #remove space, else we can't convert str to float 
    prov2 = []
    for i in prov:     
            a = i.replace(" ", "")
            prov2.append(a)

    #convert list str to float list 
    liste=([float(x) for x in prov2])
    
    #sub sum
    liste=sum(liste)
    
    return(liste)



#----------CONVERT TO CSV-----------#

def convert_csv(URSSAF,retraite_complémentaire,mutuelle_prévoyance,taxe_app_et_formation_pro):
    
    df = pd.DataFrame(columns=['mois','urssaf','retraite_complementaire','mutuelle','taxe_formation'])
    new = {'mois': "janvier", 'urssaf': URSSAF,'retraite_complementaire':retraite_complémentaire,'mutuelle':mutuelle_prévoyance,'taxe_formation': taxe_app_et_formation_pro}
    df =df.append(new, ignore_index=True)
    
    

    field_names = ['mois','urssaf','retraite_complementaire','mutuelle','taxe_formation']

    #with open('paie2.csv', 'a+', newline ='') as f_object:
     #   csv_writer = DictWriter(f_object,fieldnames=field_names )
      #  if f_object.tell() == 0:
       #         csv_writer.writeheader()
        #csv_writer.writerow(new) 
        #f_object.close()
    
    return df
  


#-----------MAIN----------#
def main():
    
    path = input()
    mylist =load(path)
    ursaf01_1 = ursaff1(mylist)
    ursaf01_2= ursaff2(mylist)
    ursaf01_3= ursaff3(mylist)
    ursaf01_4= ursaff4(mylist)
    ursaf01_5 = ursaff5(mylist)
    retraite = retraite1(mylist)
    mutuelle = mutuelle1(mylist)
    taxe = taxe1(mylist)
    #print(ursaf01_1,'\n',ursaf01_2,'\n',ursaf01_3,'\n',ursaf01_4,'\n',ursaf01_5,'\n',retraite,'\n',mutuelle,'\n',taxe)
    a = sub_sum(ursaf01_1)
    b = sub_sum(ursaf01_2)
    c = sub_sum(ursaf01_3)
    d = sub_sum(ursaf01_4)
    e = sub_sum(ursaf01_5)
    f = sub_sum(retraite)
    g = sub_sum(mutuelle)
    h = sub_sum(taxe)
    
    #print(a,'\n',b,'\n',c,'\n',d,'\n',e,'\n',f,'\n',g,'\n',h)
    URSSAF = a+b+c+d+e
    retraite_complémentaire= f
    mutuelle_prévoyance = g
    taxe_app_et_formation_pro = h
    nom = nom1(mylist)
    print('\nnom: ',nom)
    print('URSSAF: ', URSSAF,'\nretraite complémentaire: ',retraite_complémentaire,'\nmutuelle: ', mutuelle_prévoyance,'\ntaxe et formation: ', taxe_app_et_formation_pro)
    df = convert_csv(URSSAF,retraite_complémentaire,mutuelle_prévoyance,taxe_app_et_formation_pro)
    df.head()
    print('\n',df)
    

if __name__ == "__main__":
    main()


fiche_individuelle_A1221.pdf

nom:  Mlle      BENHALIMA      Rania
URSSAF:  35984.66 
retraite complémentaire:  2317.14 
mutuelle:  5363.299999999999 
taxe et formation:  5432.34

       mois    urssaf  retraite_complementaire  mutuelle  taxe_formation
0  janvier  35984.66                  2317.14    5363.3         5432.34


# Brouillon

In [ ]:
def load(path):
    input_text= extract_text(path)
    with open("format2.txt", "w", encoding="utf-8") as datafile:
        print(input_text, file=datafile) # this goes to the opened file.

    f = open("format2.txt", encoding = 'utf-8').read()
    mylist=f.split('\n')
    return (mylist)
